In [ ]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.test_options import TestOptions
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

opt = TestOptions().parse(save=False)
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

opt.label_feat = True
opt.model = 'ov_pix2pixHD'
opt.name = 'zalando_shape'
opt.dataroot= './datasets/vfr_data/'
opt.shape_generation = True
# opt.use_generator_last_activation = False
opt.input_nc= 20 
opt.output_nc = 20


opt.checkpoints_dir = "./checkpoints/"

opt

In [ ]:
# data_loader = CreateDataLoader(opt)
# dataset = data_loader.load_data()
augment = {}
augment['1'] = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=20),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

augment['2'] = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

test_dataset = TestDataset(opt, augment)

test_dataloader = DataLoader(test_dataset,
                              shuffle=False,
                              drop_last=False,
                              num_workers=6,
                              batch_size=opt.batchSize,
                              pin_memory=True)
dataset_size = len(test_dataset)
print('#testing images = %d' % dataset_size)

## FOR DEBUGGING 
# input_dict = {
#             'query_parse_map': A_tensor_label,
#             'ref_parse_map': B_tensor_label,
#             'query_seg_map': query_label_seg_mask,
#             'ref_seg_map': ref_label_seg_mask,
#             'query_img': A_tensor_img,
#             'ref_img': B_tensor_img,
#             'C_tensor': C_tensor
#         }
# for i in test_dataset[0].keys():
#     try:
#         print('{}-------'.format(i),test_dataset[0][i].shape)
#     except Exception as e:
#         print(e)

In [ ]:
model = create_model(opt)

In [ ]:
# torch.unique(test_dataset[0]['C_tensor'])

In [ ]:
# from matplotlib import pyplot as plt
# from PIL import Image
# from PIL import Image
# import numpy as np
# x = test_dataset[0]['C_tensor'].cpu().float().numpy()
# for i in range(20):
#     plt.show()
#     plt.imshow(Image.fromarray((x[i,:,:]).astype(np.uint8)))

In [ ]:
visualizer = Visualizer(opt)
# create website
web_dir = os.path.join(opt.results_dir, opt.name, '%s_%s' % (opt.phase, opt.which_epoch))
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt.name, opt.phase, opt.which_epoch))
for i, data in enumerate(test_dataloader):
    if i >= opt.how_many:
        break
    query_ref_mixed, generated = model.inference_forward_shape(data['query_parse_map'],data['ref_parse_map'],
                                                              data['C_tensor'])
    visuals = OrderedDict([('query', util.tensor2label(data['query_parse_map'][0], opt.label_nc)),
                           ('ref', util.tensor2label(data['ref_parse_map'][0], opt.label_nc)),
                           ('query_ref_mixed', util.tensor2label(query_ref_mixed.data[0], opt.label_nc)),
                           ('synthesized_Simage', util.tensor2label(generated.data[0], opt.label_nc)),
                          ('synthesized_image_edgemap', util.tensor2edgemap(torch.softmax(generated.data[0],dim=0)))])
    img_path = data['path']
    print('process image... %s' % img_path)
    visualizer.save_images(webpage, visuals, img_path)

webpage.save()

In [ ]:
# import numpy as np
# np.unique(visuals['synthesized_image_edgemap'])

In [ ]:
# from PIL import Image
# test = Image.open('./datasets/fashion_compatibility/test_query_ref_label/0_N00098_synthesized_image_edgemap.jpg')
# print(np.unique(np.array(test)))
# test = Image.open('./results/fashion_compatibility_shape/test_latest/images/0_N00098_synthesized_image_edgemap.png')
# np.unique(np.array(test))

In [ ]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.test_options import TestOptions
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

opt = TestOptions().parse(save=False)
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

opt.label_feat = True
opt.model = 'ov_pix2pixHD'
opt.name = 'zalando_appearance'
opt.dataroot= './datasets/vfr_data/'
# opt.dataroot= './datasets/fashion_compatibility/'
opt.appearance_generation = True
opt.use_generator_last_activation = True
opt.input_nc= 20 
opt.output_nc = 3

In [ ]:
# data_loader = CreateDataLoader(opt)
# dataset = data_loader.load_data()
augment = {}
augment['1'] = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=20),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

augment['2'] = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

test_dataset = TestDataset(opt, augment)

test_dataloader = DataLoader(test_dataset,
                              shuffle=False,
                              drop_last=False,
                              num_workers=6,
                              batch_size=opt.batchSize,
                              pin_memory=True)
dataset_size = len(test_dataset)
print('#testing images = %d' % dataset_size)

# # FOR DEBUGGING 
# input_dict = {
#             'query_parse_map': A_tensor_label,
#             'ref_parse_map': B_tensor_label,
#             'query_seg_map': query_label_seg_mask,
#             'ref_seg_map': ref_label_seg_mask,
#             'query_img': A_tensor_img,
#             'ref_img': B_tensor_img,
#             'C_tensor': C_tensor
#         }
# for i in test_dataset[0].keys():
#     try:
#         print('{}-------'.format(i),test_dataset[0][i].shape)
#     except Exception as e:
#         print(e)

In [ ]:
model = create_model(opt)

In [ ]:
visualizer = Visualizer(opt)
# create website
web_dir = os.path.join(opt.results_dir, opt.name, '%s_%s' % (opt.phase, opt.which_epoch))
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt.name, opt.phase, opt.which_epoch))
for i, data in enumerate(test_dataloader):
    if i >= opt.how_many:
        break
    generated = model.inference_forward_appearance(data['query_img'],data['query_parse_map'],
                                                              data['query_seg_map'],data['ref_img'],
                                                              data['ref_parse_map'],data['ref_seg_map'],
                                                              data['C_tensor'])
    
    visuals = OrderedDict([('query_img', util.tensor2im(data['query_img'][0])),
                           ('ref_image', util.tensor2im(data['ref_img'][0])),
                           ('generated_parse_map', util.tensor2label(data['C_tensor'][0], opt.label_nc)),
                           ('synthesized_image', util.tensor2im(generated.data[0]))])
    
    img_path = data['path']
    print('process image... %s' % img_path)
    visualizer.save_images(webpage, visuals, img_path)

webpage.save()

In [ ]:
app_feature_vectors_query = torch.zeros((20,30)).float().cuda()

In [ ]:
app_feature_vectors_query[0,0] = 1

In [1]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.test_options import TestOptions
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

opt = TestOptions().parse(save=False)
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

opt.model = 'ov_pix2pixHD'
opt.name = 'fashion_compatibility_appearance'
opt.dataroot= './datasets/online_data/'
# opt.dataroot= './datasets/fashion_compatibility/'
opt.appearance_generation = True
opt.use_generator_last_activation = True
opt.input_nc= 20 
opt.output_nc = 3

------------ Options -------------
appearance_generation: False
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: /home/dataset/
densepose_nc: 3
display_winsize: 512
engine: None
export_onnx: None
feat_num: 30
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
input_nc: 20
instance_feat: False
isTrain: False
label_feat: False
label_nc: 20
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: ov_pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: vfr
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
norm: instance
ntest: inf
onnx: None
output_nc: 20
phase: test
resize_or_crop: scale_width
results_dir: ./results/
serial_batches: False
shape_generation: False
tf_log: False
use_dropout: False
use_encoded_image: False
use_generator_last_activation: False
verbose: False

In [2]:
# data_loader = CreateDataLoader(opt)
# dataset = data_loader.load_data()
augment = {}
augment['1'] = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=20),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

augment['2'] = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]

test_dataset = TestDataset(opt, augment)

test_dataloader = DataLoader(test_dataset,
                              shuffle=False,
                              drop_last=False,
                              num_workers=6,
                              batch_size=opt.batchSize,
                              pin_memory=True)
dataset_size = len(test_dataset)
print('#testing images = %d' % dataset_size)

# # FOR DEBUGGING 
# input_dict = {
#             'query_parse_map': A_tensor_label,
#             'ref_parse_map': B_tensor_label,
#             'query_seg_map': query_label_seg_mask,
#             'ref_seg_map': ref_label_seg_mask,
#             'query_img': A_tensor_img,
#             'ref_img': B_tensor_img,
#             'C_tensor': C_tensor
#         }
# for i in test_dataset[0].keys():
#     try:
#         print('{}-------'.format(i),test_dataset[0][i].shape)
#     except Exception as e:
#         print(e)

#testing images = 10


In [3]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.train_options import TrainOptions
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import time

opt = TrainOptions().parse(save=False)
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip

opt.model = 'ov_pix2pixHD_online'
opt.name = 'fashion_compatibility_online'
opt.dataroot= './datasets/online_data/'
# opt.dataroot= './datasets/fashion_compatibility/'
opt.appearance_generation = True
opt.use_generator_last_activation = True
opt.input_nc= 20 
opt.output_nc = 3
opt.load_pretrain = './checkpoints/zalando_appearance/'

------------ Options -------------
appearance_generation: False
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
dataroot: /home/dataset/
debug: False
densepose_nc: 3
display_freq: 100
display_winsize: 512
feat_num: 30
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 20
instance_feat: False
isTrain: True
label_feat: False
label_nc: 20
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.001
max_dataset_size: inf
model: ov_pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: vfr
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_ce_loss: False
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 20
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: scale_width
save_epoch_freq: 

In [4]:
model = create_model(opt)

--------- model used --------- ov_pix2pixHD_online
GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(50, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14)

./checkpoints/zalando_appearance/latest_net_D.pth
Pretrained network D has fewer layers; The following are not initialized:
['scale0_layer0', 'scale0_layer1', 'scale0_layer2', 'scale0_layer3', 'scale0_layer4', 'scale1_layer0', 'scale1_layer1', 'scale1_layer2', 'scale1_layer3', 'scale1_layer4']
./checkpoints/zalando_appearance/latest_net_E.pth


In [5]:
# Training Visualizer
visualizer = Visualizer(opt)

# Optimizers
optimizer_G, optimizer_D = model.module.optimizer_G, model.module.optimizer_D

create web directory ./checkpoints/fashion_compatibility_online/web...


In [6]:
prev_j = 0
index_check = 10
for i, data in enumerate(test_dataloader, start=0):
    for j in range(0,2000):
        save_fake = True if j % index_check == 0 else False
        losses, generated = model(data['query_img'],data['query_parse_map'],
                                                    data['query_seg_map'],data['ref_img'],
                                                    data['ref_parse_map'],data['ref_seg_map'],
                                                    data['C_tensor'],infer=save_fake)

        # sum per device losses
        losses = [torch.mean(x) if not isinstance(x, int) else x for x in losses]
        loss_dict = dict(zip(model.module.loss_names, losses))

        # calculate final loss scalar
        loss_ref = (loss_dict['G_GAN_REF'] + loss_dict['G_VGG'])
        loss_query = loss_dict['G_GAN_QUERY']
        loss_online = loss_ref + loss_query

        ############### Backward Pass ####################
        # update generator weights
        optimizer_G.zero_grad()
        loss_online.backward()
        optimizer_G.step()
        
        print(f'{i}th image, {j}th step, loss_ref {loss_ref}, loss_query {loss_query}, loss_online {loss_online}')
        
        ############## Display results and errors ##########
        if save_fake:
            visuals = OrderedDict([('query_img', util.tensor2im(data['query_img'][0])),
                               ('ref_image', util.tensor2im(data['ref_img'][0])),
                               ('synthesized_image', util.tensor2im(generated.data[0]))])
            visualizer.display_current_results(visuals, (j // index_check) + prev_j, i)
    prev_j = (j // index_check)*(i+1)

/home/diominor/Workspace/vfr/Image-Based-Virtual-Try-on-Network-from-Unpaired-Data/inference/models/ov_pix2pixHD_model_online.py:200: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  indices = (selected_seg_mask_tensor == int(


0th image, 0th step, loss_ref 12.971076011657715, loss_query 4.450554370880127, loss_online 17.421630859375
0th image, 1th step, loss_ref 11.158342361450195, loss_query 4.351542949676514, loss_online 15.509885787963867
0th image, 2th step, loss_ref 9.777921676635742, loss_query 4.433220863342285, loss_online 14.211142539978027
0th image, 3th step, loss_ref 8.937857627868652, loss_query 4.409516334533691, loss_online 13.347373962402344
0th image, 4th step, loss_ref 8.389012336730957, loss_query 4.421329021453857, loss_online 12.810340881347656
0th image, 5th step, loss_ref 8.208614349365234, loss_query 4.550348281860352, loss_online 12.758962631225586
0th image, 6th step, loss_ref 7.7180585861206055, loss_query 4.558378219604492, loss_online 12.276436805725098
0th image, 7th step, loss_ref 7.475846290588379, loss_query 4.482411861419678, loss_online 11.958257675170898
0th image, 8th step, loss_ref 7.300691604614258, loss_query 4.603310585021973, loss_online 11.90400218963623
0th image, 

0th image, 75th step, loss_ref 4.63020658493042, loss_query 4.879257678985596, loss_online 9.509464263916016
0th image, 76th step, loss_ref 4.619541645050049, loss_query 4.869450569152832, loss_online 9.488992691040039
0th image, 77th step, loss_ref 4.626879692077637, loss_query 4.879951477050781, loss_online 9.506831169128418
0th image, 78th step, loss_ref 4.609847545623779, loss_query 4.8717756271362305, loss_online 9.481622695922852
0th image, 79th step, loss_ref 4.600499153137207, loss_query 4.878787040710449, loss_online 9.479286193847656
0th image, 80th step, loss_ref 4.595876216888428, loss_query 4.87169885635376, loss_online 9.467575073242188
0th image, 81th step, loss_ref 4.616490840911865, loss_query 4.884742736816406, loss_online 9.50123405456543
0th image, 82th step, loss_ref 4.700150966644287, loss_query 4.8602728843688965, loss_online 9.560423851013184
0th image, 83th step, loss_ref 4.936013698577881, loss_query 4.91644287109375, loss_online 9.852457046508789
0th image, 8

0th image, 150th step, loss_ref 4.513915061950684, loss_query 4.871352195739746, loss_online 9.38526725769043
0th image, 151th step, loss_ref 4.512032508850098, loss_query 4.8845415115356445, loss_online 9.396574020385742
0th image, 152th step, loss_ref 4.510570049285889, loss_query 4.8783769607543945, loss_online 9.388946533203125
0th image, 153th step, loss_ref 4.50909423828125, loss_query 4.879678249359131, loss_online 9.388772964477539
0th image, 154th step, loss_ref 4.511314868927002, loss_query 4.883157730102539, loss_online 9.394472122192383
0th image, 155th step, loss_ref 4.5116119384765625, loss_query 4.876925945281982, loss_online 9.388538360595703
0th image, 156th step, loss_ref 4.519014835357666, loss_query 4.888086318969727, loss_online 9.407100677490234
0th image, 157th step, loss_ref 4.531975269317627, loss_query 4.872459411621094, loss_online 9.404434204101562
0th image, 158th step, loss_ref 4.559505462646484, loss_query 4.896223068237305, loss_online 9.455728530883789


0th image, 224th step, loss_ref 4.479660987854004, loss_query 4.881945610046387, loss_online 9.36160659790039
0th image, 225th step, loss_ref 4.477940082550049, loss_query 4.884443283081055, loss_online 9.362382888793945
0th image, 226th step, loss_ref 4.478982925415039, loss_query 4.885472297668457, loss_online 9.364455223083496
0th image, 227th step, loss_ref 4.4781107902526855, loss_query 4.8831400871276855, loss_online 9.361250877380371
0th image, 228th step, loss_ref 4.478537559509277, loss_query 4.884857654571533, loss_online 9.363395690917969
0th image, 229th step, loss_ref 4.480807781219482, loss_query 4.885738372802734, loss_online 9.366546630859375
0th image, 230th step, loss_ref 4.482598304748535, loss_query 4.88205623626709, loss_online 9.364654541015625
0th image, 231th step, loss_ref 4.50245475769043, loss_query 4.8905839920043945, loss_online 9.393038749694824
0th image, 232th step, loss_ref 4.507644176483154, loss_query 4.876583099365234, loss_online 9.384227752685547
0

...

1th image, 1187th step, loss_ref 4.411858558654785, loss_query 5.0800065994262695, loss_online 9.491865158081055
1th image, 1188th step, loss_ref 4.412705421447754, loss_query 5.080862045288086, loss_online 9.49356746673584
1th image, 1189th step, loss_ref 4.411618232727051, loss_query 5.079562187194824, loss_online 9.491180419921875
1th image, 1190th step, loss_ref 4.41536283493042, loss_query 5.080804824829102, loss_online 9.49616813659668
1th image, 1191th step, loss_ref 4.414233207702637, loss_query 5.078897476196289, loss_online 9.493130683898926
1th image, 1192th step, loss_ref 4.420889854431152, loss_query 5.0808210372924805, loss_online 9.501710891723633
1th image, 1193th step, loss_ref 4.417482376098633, loss_query 5.078843593597412, loss_online 9.496326446533203
1th image, 1194th step, loss_ref 4.421545505523682, loss_query 5.080705642700195, loss_online 9.502250671386719
1th image, 1195th step, loss_ref 4.415643215179443, loss_query 5.078320503234863, loss_online 9.493963241

1th image, 1261th step, loss_ref 4.414497375488281, loss_query 5.080463886260986, loss_online 9.49496078491211
1th image, 1262th step, loss_ref 4.415439128875732, loss_query 5.078369617462158, loss_online 9.49380874633789
1th image, 1263th step, loss_ref 4.421977996826172, loss_query 5.081876754760742, loss_online 9.503854751586914
1th image, 1264th step, loss_ref 4.425802230834961, loss_query 5.077784061431885, loss_online 9.503585815429688
1th image, 1265th step, loss_ref 4.432834625244141, loss_query 5.081513404846191, loss_online 9.514348030090332
1th image, 1266th step, loss_ref 4.4323320388793945, loss_query 5.080574035644531, loss_online 9.512906074523926
1th image, 1267th step, loss_ref 4.431281566619873, loss_query 5.080666542053223, loss_online 9.511947631835938
1th image, 1268th step, loss_ref 4.431694030761719, loss_query 5.080418586730957, loss_online 9.512112617492676
1th image, 1269th step, loss_ref 4.42979621887207, loss_query 5.083741188049316, loss_online 9.5135374069

1th image, 1335th step, loss_ref 4.4219279289245605, loss_query 5.0757527351379395, loss_online 9.4976806640625
1th image, 1336th step, loss_ref 4.428534984588623, loss_query 5.0834126472473145, loss_online 9.511947631835938
1th image, 1337th step, loss_ref 4.419185161590576, loss_query 5.0784077644348145, loss_online 9.49759292602539
1th image, 1338th step, loss_ref 4.420326232910156, loss_query 5.0791473388671875, loss_online 9.499473571777344
1th image, 1339th step, loss_ref 4.426990985870361, loss_query 5.083410739898682, loss_online 9.510401725769043
1th image, 1340th step, loss_ref 4.412146091461182, loss_query 5.0771636962890625, loss_online 9.489309310913086
1th image, 1341th step, loss_ref 4.4194793701171875, loss_query 5.083374977111816, loss_online 9.502854347229004
1th image, 1342th step, loss_ref 4.415062427520752, loss_query 5.081644058227539, loss_online 9.496706008911133
1th image, 1343th step, loss_ref 4.415887355804443, loss_query 5.081398010253906, loss_online 9.4972

1th image, 1409th step, loss_ref 4.414599418640137, loss_query 5.077850341796875, loss_online 9.492449760437012
1th image, 1410th step, loss_ref 4.413027286529541, loss_query 5.08283805847168, loss_online 9.495864868164062
1th image, 1411th step, loss_ref 4.416032791137695, loss_query 5.079855918884277, loss_online 9.495888710021973
1th image, 1412th step, loss_ref 4.413851261138916, loss_query 5.07987117767334, loss_online 9.493722915649414
1th image, 1413th step, loss_ref 4.415177345275879, loss_query 5.0795464515686035, loss_online 9.49472427368164
1th image, 1414th step, loss_ref 4.412493705749512, loss_query 5.080678939819336, loss_online 9.493172645568848
1th image, 1415th step, loss_ref 4.410617828369141, loss_query 5.0783867835998535, loss_online 9.489004135131836
1th image, 1416th step, loss_ref 4.409857273101807, loss_query 5.0811238288879395, loss_online 9.490981101989746
1th image, 1417th step, loss_ref 4.405666828155518, loss_query 5.079097747802734, loss_online 9.48476409

KeyboardInterrupt: 

In [ ]:
# Test exercises

In [ ]:
import numpy as np
from PIL import Image

In [ ]:
image_numpy = test_dataset[2]['ref_img'].cpu().float().numpy()
image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
image_numpy = image_numpy.astype(np.uint8)
Image.fromarray(image_numpy)

In [ ]:
org_seg_map = test_dataset[2]['ref_parse_map'].clone()

# unnormalize it 
unnormalize_seg_map = org_seg_map.cpu().float().numpy()
unnormalize_seg_map = (np.transpose(unnormalize_seg_map, (1, 2, 0)) + 1) / 2.0
unnormalize_seg_map = unnormalize_seg_map.astype(np.uint8)
np.unique(unnormalize_seg_map)

# filter the top section
filter_part = unnormalize_seg_map[:, :, 5] + unnormalize_seg_map[:, :, 6] + unnormalize_seg_map[ :, :,7]  # 5,6,7,
filter_part = filter_part > 0
filter_part = np.expand_dims(filter_part,-1)


image_numpy = test_dataset[2]['ref_img'].cpu().float().numpy()
image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
image_numpy = image_numpy.astype(np.uint8)
source_img = image_numpy * filter_part # other part normalized 0 --> 127 gray
Image.fromarray(source_img)

In [ ]:
ref_parse_map_online = torch.zeros((20, 512, 256)).float().cuda()
for num_seg_channel in range(20):
    if 4 < num_seg_channel < 8:
        ref_parse_map_online[num_seg_channel,:,:] = test_dataset[2]['ref_parse_map'][num_seg_channel,:,:]

In [ ]:
ref_parse_map_online = (ref_parse_map_online + 1)/2
filter_part = ref_parse_map_online[5, :, :] + ref_parse_map_online[6, :, :] + ref_parse_map_online[7, :, :]  # 5,6,7,
filter_part = filter_part > 0
filter_part = torch.unsqueeze(filter_part,0).float().cuda()
src = test_dataset[2]['ref_img'].float().cuda()
src = (src + 1)/2
source_img = src * filter_part # other part normalized 0 --> 127 gray
test = source_img.clone()
image_numpy = source_img.cpu().float().numpy()
image_numpy = np.transpose(image_numpy,(1, 2, 0))* 255.0
image_numpy = image_numpy.astype(np.uint8)
Image.fromarray(image_numpy)

In [ ]:
norm = transforms.Normalize((0.5,), (0.5,))
out = norm(test)
torch.unique(out)